In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from flask import Flask, request, jsonify
import numpy as np
from transformation_functions import square_root_transformation , log_transformation , Reciprocal_transformation


In [2]:
cleaned_cars_data = pd.read_csv('cleaned_cars_data.csv')

In [3]:
with open('ridge_model.pkl', "rb") as file:
    ridge_model = pickle.load(file)


In [4]:

with open('scaler_other.pkl', 'rb') as scaler_other_file:
    scaler_info_other = pickle.load(scaler_other_file)
# Access the actual scaler object from the loaded information
loaded_scaler_for_other = scaler_info_other['scaler']


In [5]:
with open('Reciprocal_transformation_info.pkl', 'rb') as file:
    load_Reciprocal_transformation_info = pickle.load(file)
    
loaded_reciprocal_transform = load_Reciprocal_transformation_info['transformation_function']


In [6]:
with open('root_transformation_info.pkl', 'rb') as file:
    load_root_transformation_info = pickle.load(file)
    
loaded_root_transform = load_root_transformation_info['transformation_function']


In [7]:
with open('scaler_price.pkl', 'rb') as scaler_price_file:
    scaler_info_price = pickle.load(scaler_price_file)
# Access the actual scaler object from the loaded information
loaded_scaler_for_price =  scaler_info_price['scaler']


In [8]:
with open('inverse_transform_info_price.pkl' , 'rb') as inverse_transform_info_price:
    inverse_transform_info_price = pickle.load(inverse_transform_info_price)
# Access the actual scaler object from the loaded information
loaded_inverse_transform_info_price =  inverse_transform_info_price['transformation_function']


In [9]:

def Reciprocal_transformation(feature_to_transform):
    feature_reciprocal_transformation = 1 / feature_to_transform
    return feature_reciprocal_transformation


def min_max_normalization(dataframe , features_to_normalize): #must be  list 
    scaler = MinMaxScaler()
    dataframe[features_to_normalize] = scaler.fit_transform( dataframe[features_to_normalize])
    return  dataframe

def Label_Encoder(The_feature):
    label_encoder = LabelEncoder()
    The_feature_encoded = label_encoder.fit_transform(The_feature)
    return The_feature_encoded

def OneHot_Encoder(data, column):
    onehot_encoded_columns = pd.get_dummies(data[column], prefix=f'{column}-')
    data = pd.concat([data, onehot_encoded_columns], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data


reciprocal_cols = ['passengers']
min_max_cols = ['model', 'motor_power', 'car_speedometer', 'passengers', 'ex_owners', 'Number_of_Additions']

def preprocess_data(input_data, loaded_scaler , loaded_Reciprocal_transformation):
    # Reciprocal transformation
    input_data[reciprocal_cols] = loaded_Reciprocal_transformation(input_data[reciprocal_cols])

    # Min-Max normalization
    input_data[min_max_cols] = loaded_scaler.transform(input_data[min_max_cols])

    return input_data



In [38]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the data from the request
        data = request.get_json(force=True)
        features_to_modify = ['Name', 'color', 'fuel_type', 'origin_car', 'lime_type']
        features_that_have_two_values = ['car_license', 'glass']
        updated_data = {}
        for key, value in data.items():
            if key in features_to_modify:
                updated_key = f'{key}-_{value}'  # Assuming you want to concatenate the key and value
                updated_data[updated_key] = 1
            elif key in features_that_have_two_values:
                # Modify values for features with two values
                if key == 'car_license' and value == 'نمرة صفراء':
                    updated_data[key] = 1
                else:
                    updated_data[key] = 0

                if key == 'glass' and value == 'يدوي':
                    updated_data[key] = 1
                else:
                    updated_data[key] = 0
            else:
                updated_data[key] = value
                #it is ready  
        data_2.update(pd.DataFrame([updated_data]))
        input_data = preprocess_data(data_2 , loaded_scaler_for_other , loaded_reciprocal_transform) # 
        # Make predictions
        prediction = ridge_model.predict(input_data)
        prediction_2d = prediction.reshape(-1, 1)
        
        #---------------------------------------------------------------------------------------------------
        # there is may some problem in the inverse process in target *******************
        #---------------------------------------------------------------------------------------------------
        inverse_normalized_predictions = prediction_2d * (393.70039370059055 - 34.64101615137755) + 34.64101615137755

        # Inverse Transform for Root Transformation
        prediction_without_root =  np.power(inverse_normalized_predictions, 2) 

        # Inverse Transform 
        #prediction_original_scale = loaded_inverse_transform_info_price(prediction_without_root)

        # Now, 'prediction_original_scale' contains the predictions in their original scale
        #print(prediction_original_scale)

        response = {'prediction For Price': prediction_2d[0].tolist()}
        return jsonify(response)


    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [14]:
data = pd.read_csv('cars_data.csv')

In [15]:
data

,اسم السيارة,موديل السيارة,سعر السيارة,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,...,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,حالة الإعلان,تاريخ نشر الإعلان,تاريخ إنتهاء الإعلان,Number_of_Additions,فتحة السقف,الدفع
0,كيا اوبتيما,موديل سنة 2014,100000 شيكل,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,نقدا فقط,للبيع فقط,يد اولى,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2017-08-16,2017-09-15,7.0,1.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,كيا سورينتو,موديل سنة 2007,60000 شيكل,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500.0,...,إمكانية التقسيط,للبيع أو التبديل,2,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2017-08-16,2017-09-30,7.0,1.0,NaN
4,هونداي افانتي,موديل سنة 2006,43500 شيكل,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600.0,...,نقدا فقط,للبيع فقط,NaN,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2017-08-19,2017-09-18,7.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983,مرسيدس C220,موديل سنة 2009,90000 شيكل,فيراني,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200.0,...,نقدا فقط,للبيع فقط,يد ثانية,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2022-05-07,2022-07-06,7.0,1.0,دفع خلفي
6984,نيسان كشكاي,موديل سنة 2014,95000 شيكل,فيراني,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600.0,...,إمكانية التقسيط,للبيع فقط,يد اولا,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقففرش ...,إعلان منتهي,2022-05-07,2022-06-06,6.0,1.0,دفع أمامي
6985,سكودا اوكتافيا,موديل سنة 2020,160000 شيكل,أبيض,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,إمكانية التقسيط,للبيع أو التبديل,0,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2022-05-08,2022-07-07,7.0,0.0,دفع أمامي
6986,سكودا سوبيرب,موديل سنة 2018,119000 شيكل,أسود,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,نقدا فقط,للبيع فقط,0,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2022-05-08,2022-06-12,6.0,0.0,دفع أمامي


In [16]:
sample_row

NameError: name 'sample_row' is not defined

In [17]:
data_2 = cleaned_cars_data.copy()

In [18]:
data_2 = pd.DataFrame(0, index=[0], columns=data_2.columns)


In [19]:
data_2

,model,Price(NIS),car_license,glass,motor_power,car_speedometer,passengers,ex_owners,Number_of_Additions,sunroof,...,fuel_type-_هايبرد,origin_car-_تأجير,origin_car-_تجاري,origin_car-_تدريب سياقة,origin_car-_حكومي,origin_car-_خصوصي,origin_car-_عمومي,lime_type-_اوتوماتيك,lime_type-_عادي,lime_type-_نصف اوتوماتيك
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
 data_2.drop('Price(NIS)', axis=1, inplace=True)

In [21]:
 data_2

,model,car_license,glass,motor_power,car_speedometer,passengers,ex_owners,Number_of_Additions,sunroof,Name-_أفيكو 50c18دبل كابينه,...,fuel_type-_هايبرد,origin_car-_تأجير,origin_car-_تجاري,origin_car-_تدريب سياقة,origin_car-_حكومي,origin_car-_خصوصي,origin_car-_عمومي,lime_type-_اوتوماتيك,lime_type-_عادي,lime_type-_نصف اوتوماتيك
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
filtered_data = cleaned_cars_data[cleaned_cars_data['Name-_هونداي افانتي'] == 1]
filtered_data

,model,Price(NIS),car_license,glass,motor_power,car_speedometer,passengers,ex_owners,Number_of_Additions,sunroof,...,fuel_type-_هايبرد,origin_car-_تأجير,origin_car-_تجاري,origin_car-_تدريب سياقة,origin_car-_حكومي,origin_car-_خصوصي,origin_car-_عمومي,lime_type-_اوتوماتيك,lime_type-_عادي,lime_type-_نصف اوتوماتيك
2,0.722222,0.484392,0,0,0.428571,0.226791,0.40,0.2,1.000000,0.0,...,0,0,0,0,0,1,0,1,0,0
29,0.722222,0.474289,0,0,0.428571,0.399985,0.40,0.4,0.000000,0.0,...,0,0,0,0,0,1,0,1,0,0
168,0.703704,0.494322,0,0,0.428571,0.000000,0.40,0.6,1.000000,1.0,...,0,0,0,0,0,1,0,1,0,0
288,0.722222,0.474289,0,0,0.428571,0.003998,0.40,0.4,0.857143,0.0,...,0,0,0,0,0,1,0,1,0,0
304,0.722222,0.460534,0,0,0.428571,0.453582,0.25,0.4,0.857143,0.0,...,0,0,0,0,0,1,0,1,0,0
370,0.722222,0.439239,0,0,0.428571,0.000399,0.40,0.6,0.571429,0.0,...,0,0,0,0,0,1,0,1,0,0
765,0.722222,0.460534,0,0,0.428571,0.550779,0.40,0.4,0.857143,0.0,...,0,0,0,0,0,1,0,1,0,0
864,0.703704,0.460534,0,0,0.428571,0.000648,0.40,0.8,1.000000,0.0,...,0,0,0,0,0,1,0,1,0,0
993,0.722222,0.474289,0,0,0.428571,0.226791,0.40,0.2,1.000000,0.0,...,0,0,0,0,0,1,0,0,1,0
1371,0.722222,0.500851,0,0,0.428571,0.388785,0.40,0.2,0.285714,0.0,...,0,0,0,0,0,1,0,1,0,0


In [23]:
data = pd.read_csv('cars_data.csv')

In [24]:
data

,اسم السيارة,موديل السيارة,سعر السيارة,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,...,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,حالة الإعلان,تاريخ نشر الإعلان,تاريخ إنتهاء الإعلان,Number_of_Additions,فتحة السقف,الدفع
0,كيا اوبتيما,موديل سنة 2014,100000 شيكل,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,نقدا فقط,للبيع فقط,يد اولى,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2017-08-16,2017-09-15,7.0,1.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,كيا سورينتو,موديل سنة 2007,60000 شيكل,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500.0,...,إمكانية التقسيط,للبيع أو التبديل,2,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2017-08-16,2017-09-30,7.0,1.0,NaN
4,هونداي افانتي,موديل سنة 2006,43500 شيكل,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600.0,...,نقدا فقط,للبيع فقط,NaN,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2017-08-19,2017-09-18,7.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983,مرسيدس C220,موديل سنة 2009,90000 شيكل,فيراني,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200.0,...,نقدا فقط,للبيع فقط,يد ثانية,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقفجنطا...,إعلان منتهي,2022-05-07,2022-07-06,7.0,1.0,دفع خلفي
6984,نيسان كشكاي,موديل سنة 2014,95000 شيكل,فيراني,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600.0,...,إمكانية التقسيط,للبيع فقط,يد اولا,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDفتحة سقففرش ...,إعلان منتهي,2022-05-07,2022-06-06,6.0,1.0,دفع أمامي
6985,سكودا اوكتافيا,موديل سنة 2020,160000 شيكل,أبيض,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,إمكانية التقسيط,للبيع أو التبديل,0,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2022-05-08,2022-07-07,7.0,0.0,دفع أمامي
6986,سكودا سوبيرب,موديل سنة 2018,119000 شيكل,أسود,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000.0,...,نقدا فقط,للبيع فقط,0,مُكيّفإغلاق مركزيجهاز إنذارمسجل CDجنطات مغنيسي...,إعلان منتهي,2022-05-08,2022-06-12,6.0,0.0,دفع أمامي
